In [38]:
#importing the required library
import selenium
import pandas as pd
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException,ElementClickInterceptedException, ElementNotVisibleException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import warnings
warnings.filterwarnings('ignore')
import requests
from bs4 import BeautifulSoup

In [28]:
def get_template(search_term,page_no=1):
    """Generate a URL for search Term with page number"""
    search_term = search_term.replace(' ','+')
    template = f'https://www.amazon.in/s?k={search_term}&page={page_no}&qid=1623864210&ref=sr_pg_{page_no}'
    
    return template

In [29]:
def get_url():#the function that generated URL with the search tearm.
    iteams = ['laptops', 'Phones', 'Headphones', 'smart watches', 'Professional Cameras', 'Printers', 'scanners', 'keyboard', 'monitors', 'Home theater', 'router']
    URL = []     
    for i in iteams:
        for j in range(1,15):
            URL.append(get_template(i,j))
    return URL       

In [32]:
def get_review_url(): #the function that scraps the URL of the front page of the reviews and rattings with the search tearm.
    
    hreff = []
  
    driver = webdriver.Chrome(r"C:\Users\pranay\Desktop\chromedriver_win32\chromedriver.exe")
    driver.maximize_window()

    URL = get_url()
    for k in URL:
        driver.get(k) #Opening with the URL template

        for l in driver.find_elements(By.XPATH,"//a[@class = 'a-link-normal']"):
            hreff.append(l.get_attribute("href"))   
    return hreff

In [33]:
href = get_review_url()
len(href)

1533

In [34]:
href

['https://aax-eu.amazon.in/x/c/RNR7fYkeWN5IFKpwADev-F4AAAGEi4In8QMAAAH2AQBOL0EgICAgICAgICAgICBOL0EgICAgICAgICAgICDC1NMM/https://www.amazon.in/dp/B0B569D2Y4?pd_rd_i=B0B569D2Y4&pf_rd_p=d882f7bc-c7c5-4887-b939-1af6f91cc448&pf_rd_r=STFM35RVVTKX8NV8SW3A&pd_rd_wg=eTL6A&pd_rd_w=u2obi&pd_rd_r=d3e58499-7ed6-4094-8b23-9ad4d8402193',
 'https://aax-eu.amazon.in/x/c/RNR7fYkeWN5IFKpwADev-F4AAAGEi4In8QMAAAH2AQBOL0EgICAgICAgICAgICBOL0EgICAgICAgICAgICDC1NMM/https://www.amazon.in/dp/B0B569D2Y4?pd_rd_i=B0B569D2Y4&pf_rd_p=d882f7bc-c7c5-4887-b939-1af6f91cc448&pf_rd_r=STFM35RVVTKX8NV8SW3A&pd_rd_wg=eTL6A&pd_rd_w=u2obi&pd_rd_r=d3e58499-7ed6-4094-8b23-9ad4d8402193',
 'https://aax-eu.amazon.in/x/c/RNR7fYkeWN5IFKpwADev-F4AAAGEi4In8QMAAAH2AQBOL0EgICAgICAgICAgICBOL0EgICAgICAgICAgICDC1NMM/https://www.amazon.in/dp/B0B569D2Y4?pd_rd_i=B0B569D2Y4&pf_rd_p=d882f7bc-c7c5-4887-b939-1af6f91cc448&pf_rd_r=STFM35RVVTKX8NV8SW3A&pd_rd_wg=eTL6A&pd_rd_w=u2obi&pd_rd_r=d3e58499-7ed6-4094-8b23-9ad4d8402193',
 'https://www.amazon.in/H

In [39]:
def get_review(): #the function that scraps the URL of all the reviews and rattings
    
    driver = webdriver.Chrome(r"C:\Users\pranay\Desktop\chromedriver_win32\chromedriver.exe")
    driver.maximize_window() #Calling the Web Driver
  
    rattings = []
    review = []

    for i in href:
        driver.get(i) #Opening with the URL template
        
        try: #scraping the URL of the full review pages
            link = driver.find_element(By.XPATH,"//a[@data-hook='see-all-reviews-link-foot']")
            link = link.get_attribute("href")
            link= link.split('ref=cm')[0]
        except:
            pass     
        
        for i in range(1,5): #Scraping Reviews and Rattings
            l1= f'ref=cm_cr_getr_d_paging_btm_next_{i}?ie=UTF8&reviewerType=all_reviews&pageNumber={i}'
            l = link+ l1 
            
            
            driver.get(l)
            
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            results = soup.find_all('div',{'class' : 'a-section celwidget'})
        
            for item in results:
                    try:
                        rev = item.find('span', {'data-hook': "review-body"})
                        atag = item.find(['a'], class_ = "a-link-normal")
                        if rev.span.text.replace('\n','').strip() and atag['title'][0]:
                            review.append(rev.span.text.replace('\n','').strip())
                            rattings.append(atag['title'][0])
                        else:
                            break
                    except:
                        break
       
    driver.close()
    
    rat = pd.DataFrame({'Rattings':rattings, "Review": review})
    
    #saving the dataframe in csv
    rat.to_csv("Rattings1.csv",index=False) #Creating CSV
    
    return rat

In [40]:
Rat = get_review()
Rat

,Rattings,Review
0,1,Please don't purchase any Lenovo product.4 mon...
1,1,Please don't purchase any Lenovo product.4 mon...
2,1,Please don't purchase any Lenovo product.4 mon...
3,2,I usually research a lot before buying a lapto...
4,5,"I am a software engineer, working with Reactjs..."
...,...,...
46861,5,I have several of thes across the whole house....
46862,2,This Deco x60 has many limitations compared to...
46863,4,I was originally using deco E 4 route and was ...
46864,4,HiAlways had a problem of getting strong enoug...


In [41]:
Rat.to_csv("Rattings_scraped.csv",index=False)# saving Data frame as CSV